# Circuit learning module: Lambeq manually with SPSA and JAX

This module performs the optimization of the parametrized circuit manually compared to Lambeq's automatic QuantumTrainer class. I created this because I wanted to have more control over the optimization process and debug it better. The code is based on the workflow presented in https://github.com/CQCL/Quanthoven.

In [1]:
import warnings
import json
import os
import sys
import glob
from math import ceil
from pathlib import Path
from jax import numpy as np
from sympy import default_sort_key
import numpy
import pickle
import matplotlib.pyplot as plt

import jax
from jax import jit
from noisyopt import minimizeSPSA, minimizeCompass

from discopy.quantum import Circuit
from discopy.tensor import Tensor
from discopy.utils import loads
#from pytket.extensions.qiskit import AerBackend
#from pytket.extensions.qulacs import QulacsBackend
#from pytket.extensions.cirq import CirqStateSampleBackend
backend = None

from utils import *
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

warnings.filterwarnings('ignore')
this_folder = os.path.abspath(os.getcwd())
os.environ['TOKENIZERS_PARALLELISM'] = 'true'
#os.environ["JAX_PLATFORMS"] = "cpu"

SEED = 0

# This avoids TracerArrayConversionError from jax
Tensor.np = np

rng = numpy.random.default_rng(SEED)
numpy.random.seed(SEED)

## Read circuit data

We read the circuits from the pickled files. Select if we perform binary classification or multi-class classification. Give number of qubits to create classes:
- 1 qubits -> 2^1 = 2 classes i.e. binary classification
- 2 qubits -> 2^2 = 4 classes
- ...
- 5 qubits -> 2^5 = 32 classes, etc.

In [2]:
# Select workload
workload = "execution_time"
#workload = "cardinality"

# Select workload size
#workload_size = "small"
#workload_size = "medium"
#workload_size = "large"
workload_size = "main"

classification = 2
layers = 1
single_qubit_params = 3
n_wire_count = 1

loss = multi_class_loss
acc = multi_class_acc

if classification == 1:
    loss = bin_class_loss
    acc = bin_class_acc

# Access the selected circuits
path_name = this_folder + "//simplified-JOB-diagrams//" + workload + "//" + workload_size + "//circuits//" + str(classification) + "//" + str(layers) + "_layer//" + str(single_qubit_params) + "_single_qubit_params//" + str(n_wire_count) + "_n_wire_count//"

training_circuits_paths = glob.glob(path_name + "training//[0-9]*.p")
validation_circuits_paths = glob.glob(path_name + "validation//[0-9]*.p")
test_circuits_paths = glob.glob(path_name + "test//[0-9]*.p")

In [3]:
training_circuits = read_diagrams(training_circuits_paths)
validation_circuits = read_diagrams(validation_circuits_paths)
test_circuits = read_diagrams(test_circuits_paths)

## Read training and test data

In [4]:
training_data, test_data, validation_data = None, None, None
data_path = this_folder + "//data//" + workload + "//" + workload_size + "//"

with open(data_path + "training_data.json", "r") as inputfile:
    training_data = json.load(inputfile)['training_data']
with open(data_path + "test_data.json", "r") as inputfile:
    test_data = json.load(inputfile)['test_data']
with open(data_path + "validation_data.json", "r") as inputfile:
    validation_data = json.load(inputfile)['validation_data']

training_data_labels = create_labeled_classes(training_data, classification, workload)
test_data_labels = create_labeled_classes(test_data, classification, workload)
validation_data_labels = create_labeled_classes(validation_data, classification, workload)

## Lambeq optimizer

## Model

In [5]:
def make_pred_fn(circuits):
    # In the case we want to use other backends. 
    # Currently does not work properly.
    if backend:
        compiled_circuits1 = backend.get_compiled_circuits([c.to_tk() for c in circuits])
        circuits = [Circuit.from_tk(c) for c in compiled_circuits1]
        
    circuit_fns = [c.lambdify(*parameters) for c in circuits]
    
    def predict(params):
        outputs = Circuit.eval(*(c(*params) for c in circuit_fns), backend = backend)
        res = []
        
        for output in outputs:
            predictions = np.abs(output.array) + 1e-9
            ratio = predictions / predictions.sum()
            res.append(ratio)
            
        return np.array(res)
    return predict

## Loss function and evaluation

In [6]:
def make_cost_fn(pred_fn, labels):
    def cost_fn(params, **kwargs):
        predictions = pred_fn(params)

        cost = loss(predictions, labels) #-np.sum(labels * np.log(predictions)) / len(labels)  # binary cross-entropy loss
        costs.append(cost)

        accuracy = acc(predictions, labels) #np.sum(np.round(predictions) == labels) / len(labels) / 2  # half due to double-counting
        accuracies.append(accuracy)

        return cost

    costs, accuracies = [], []
    return cost_fn, costs, accuracies

## Minimization with noisyopt

In [7]:
def initialize_parameters(old_params, old_values, new_params):
    new_values = list(numpy.array(rng.random(len(new_params))))
    old_param_dict = {}
    for p, v in zip(old_params, old_values):
        old_param_dict[p] = v
        
    parameters = sorted(set(old_params + new_params), key=default_sort_key)
    values = []
    for p in parameters:
        if p in old_param_dict:
            values.append(old_param_dict[p])
        else:
            values.append(new_values.pop())
            
    return parameters, np.array(values)

In [8]:
EPOCHS = 4000
initial_number_of_circuits = 192
syms = {}
limit = False
all_training_keys = list(training_circuits.keys())
initial_circuit_keys = all_training_keys[:initial_number_of_circuits + 1]
current_training_circuits = {}
result_file = workload + "_" + workload_size + "_noisyopt_" + str(classification) + "_" + str(layers) + "_" + str(single_qubit_params)

for k in initial_circuit_keys:
    current_training_circuits[k] = training_circuits[k]
    
syms = get_symbols(current_training_circuits)
parameters = sorted(syms, key=default_sort_key)
if initial_number_of_circuits > 5 and os.path.exists("points//" + result_file + ".npz"):
    with open("points//" + result_file + ".npz", "rb") as f:
        print("Loading parameters from file " + result_file)
        npzfile = np.load(f)
        init_params_spsa = npzfile['arr_0']
else:
    print("Initializing new parameters")
    init_params_spsa = np.array(rng.random(len(parameters)))
result = None
run = 0

Loading parameters from file execution_time_main_noisyopt_2_1_3


In [ ]:
for i, key in enumerate(all_training_keys[initial_number_of_circuits:]):
    print("Progress: ", round((i + initial_number_of_circuits)/len(all_training_keys), 3))
    
    if len(syms) == len(get_symbols(current_training_circuits)) and i > 0:
        if i != len(all_training_keys[1:]):
            current_training_circuits[key] = training_circuits[key]
            new_parameters = sorted(get_symbols({key: training_circuits[key]}), key=default_sort_key)
            if result:
                parameters, init_params_spsa = initialize_parameters(parameters, result.x, new_parameters)
                #continue
            else:
                syms = get_symbols(current_training_circuits)
                parameters = sorted(syms, key=default_sort_key)
                init_params_spsa = np.array(rng.random(len(parameters)))
    else:
        run += 1
    
    # Select those circuits from test and validation circuits which share the parameters with the current training circuits
    current_validation_circuits = select_circuits(current_training_circuits, validation_circuits)
    current_test_circuits = select_circuits(current_training_circuits, test_circuits)
    
    if len(current_validation_circuits) == 0 or len(current_test_circuits) == 0:
        continue
    
    # Create lists with circuits and their corresponding label
    training_circuits_l, training_data_labels_l = construct_data_and_labels(current_training_circuits, training_data_labels)
    validation_circuits_l, validation_data_labels_l = construct_data_and_labels(current_validation_circuits, validation_data_labels)
    test_circuits_l, test_data_labels_l = construct_data_and_labels(current_test_circuits, test_data_labels)
    
    # Limit the number of validation and test circuits to 20% of number of the training circuits
    if limit:
        val_test_circ_size = ceil(len(current_training_circuits))
        if len(current_validation_circuits) > val_test_circ_size:
            validation_circuits_l = validation_circuits_l[:val_test_circ_size]
            validation_data_labels_l = validation_data_labels_l[:val_test_circ_size]
        if len(current_test_circuits) > val_test_circ_size:
            test_circuits_l = test_circuits_l[:val_test_circ_size]
            test_data_labels_l = test_data_labels_l[:val_test_circ_size]
    
    stats = f"Number of training circuits: {len(training_circuits_l)}   "\
        + f"Number of validation circuits: {len(validation_circuits_l)}   "\
        + f"Number of test circuits: {len(test_circuits_l)}   "\
        + f"Number of parameters in model: {len(set([sym for circuit in training_circuits_l for sym in circuit.free_symbols]))}"
    
    with open("results//" + result_file + ".txt", "a") as f:
        f.write(stats + "\n")
    
    print(stats)
    
    train_pred_fn = jit(make_pred_fn(training_circuits_l))
    dev_pred_fn = jit(make_pred_fn(validation_circuits_l))
    test_pred_fn = make_pred_fn(test_circuits_l)
    
    train_cost_fn, train_costs, train_accs = make_cost_fn(train_pred_fn, training_data_labels_l)
    dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, validation_data_labels_l)
    
    def callback_fn(xk):
        #print(xk)
        valid_loss = dev_cost_fn(xk)
        train_loss = numpy.around(min(float(train_costs[-1]), float(train_costs[-2])), 4)
        train_acc = numpy.around(min(float(train_accs[-1]), float(train_accs[-2])), 4)
        valid_acc = numpy.around(float(dev_accs[-1]), 4)
        iters = int(len(train_accs)/2)
        if iters % 200 == 0:
            info = f"Epoch: {iters}   "\
            + f"train/loss: {train_loss}   "\
            + f"valid/loss: {numpy.around(float(valid_loss), 4)}   "\
            + f"train/acc: {train_acc}   "\
            + f"valid/acc: {valid_acc}"
        
            with open("results//" + result_file + ".txt", "a") as f:
                f.write(info + "\n")
                
            print(info, file=sys.stderr)
        return valid_loss
    
    a_value = 0.0053
    c_value = 0.0185
            
    train_cost_fn, train_costs, train_accs = make_cost_fn(train_pred_fn, training_data_labels_l)
    dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, validation_data_labels_l)

    result = minimizeSPSA(train_cost_fn, x0=init_params_spsa, a = a_value, c = c_value, niter=EPOCHS, callback=callback_fn)
    #result = minimizeCompass(train_cost_fn, x0=init_params_spsa, redfactor=2.0, deltainit=1.0, deltatol=0.001, feps=1e-15, errorcontrol=True, funcNinit=30, funcmultfactor=2.0, paired=True, alpha=0.05, callback=callback_fn)

    figure_path = this_folder + "//results//" + result_file + ".png"
    visualize_result_noisyopt(result, make_cost_fn, test_pred_fn, test_data_labels_l, train_costs, train_accs, dev_costs, dev_accs, figure_path, result_file)
    
    run += 1
    #EPOCHS += 100
    syms = get_symbols(current_training_circuits)
    
    # Extend for the next optimization round
    current_training_circuits[key] = training_circuits[key]
    new_parameters = sorted(get_symbols({key: training_circuits[key]}), key=default_sort_key)
    parameters, init_params_spsa = initialize_parameters(parameters, result.x, new_parameters)

Progress:  0.402
Number of training circuits: 180   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267


Epoch: 200   train/loss: 56.1127   valid/loss: 42.6275   train/acc: 0.3444   valid/acc: 0.3009
Epoch: 400   train/loss: 53.0632   valid/loss: 42.7125   train/acc: 0.3667   valid/acc: 0.354
Epoch: 600   train/loss: 50.8311   valid/loss: 40.8578   train/acc: 0.4333   valid/acc: 0.292
Epoch: 800   train/loss: 48.9836   valid/loss: 40.509   train/acc: 0.4778   valid/acc: 0.3451
Epoch: 1000   train/loss: 46.6181   valid/loss: 41.5653   train/acc: 0.5167   valid/acc: 0.3097
Epoch: 1200   train/loss: 46.1221   valid/loss: 42.4933   train/acc: 0.4889   valid/acc: 0.2655
Epoch: 1400   train/loss: 45.3097   valid/loss: 43.212   train/acc: 0.4944   valid/acc: 0.2655
Epoch: 1600   train/loss: 44.0521   valid/loss: 42.8179   train/acc: 0.5222   valid/acc: 0.292
Epoch: 1800   train/loss: 43.3526   valid/loss: 43.017   train/acc: 0.5333   valid/acc: 0.2832
Epoch: 2000   train/loss: 42.7287   valid/loss: 42.979   train/acc: 0.5278   valid/acc: 0.3097
Epoch: 2200   train/loss: 42.2935   valid/loss: 43.

Test accuracy: 0.42342342342342343
Progress:  0.404
Number of training circuits: 181   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267


Epoch: 200   train/loss: 62.1087   valid/loss: 39.0861   train/acc: 0.2873   valid/acc: 0.3009
Epoch: 400   train/loss: 58.3461   valid/loss: 41.7711   train/acc: 0.3757   valid/acc: 0.2655
Epoch: 600   train/loss: 54.1501   valid/loss: 42.245   train/acc: 0.3978   valid/acc: 0.292
Epoch: 800   train/loss: 51.5246   valid/loss: 44.7453   train/acc: 0.4365   valid/acc: 0.292
Epoch: 1000   train/loss: 49.8899   valid/loss: 44.6336   train/acc: 0.4696   valid/acc: 0.3274
Epoch: 1200   train/loss: 48.5579   valid/loss: 44.7666   train/acc: 0.5028   valid/acc: 0.292
Epoch: 1400   train/loss: 46.8613   valid/loss: 44.0649   train/acc: 0.5249   valid/acc: 0.292
Epoch: 1600   train/loss: 45.7103   valid/loss: 44.7376   train/acc: 0.5359   valid/acc: 0.292
Epoch: 1800   train/loss: 45.5086   valid/loss: 43.0275   train/acc: 0.5635   valid/acc: 0.3186
Epoch: 2000   train/loss: 45.4046   valid/loss: 43.299   train/acc: 0.5746   valid/acc: 0.3186
Epoch: 2200   train/loss: 45.0735   valid/loss: 42.

Test accuracy: 0.44144144144144143
Progress:  0.406
Number of training circuits: 182   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267


...

Test accuracy: 0.4774774774774775
Progress:  0.424
Number of training circuits: 190   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267


Epoch: 200   train/loss: 61.2019   valid/loss: 37.9019   train/acc: 0.3158   valid/acc: 0.3451
Epoch: 400   train/loss: 66.1422   valid/loss: 39.9366   train/acc: 0.2474   valid/acc: 0.2832
Epoch: 600   train/loss: 57.0191   valid/loss: 41.8921   train/acc: 0.4316   valid/acc: 0.292
Epoch: 800   train/loss: 54.5762   valid/loss: 43.2819   train/acc: 0.4579   valid/acc: 0.3274
Epoch: 1000   train/loss: 53.9979   valid/loss: 38.5813   train/acc: 0.4684   valid/acc: 0.3717
Epoch: 1200   train/loss: 51.6723   valid/loss: 41.1589   train/acc: 0.5105   valid/acc: 0.3628
Epoch: 1400   train/loss: 51.8014   valid/loss: 40.1893   train/acc: 0.4737   valid/acc: 0.354
Epoch: 1600   train/loss: 52.2301   valid/loss: 39.9544   train/acc: 0.5211   valid/acc: 0.3717
Epoch: 1800   train/loss: 51.7872   valid/loss: 40.4   train/acc: 0.5105   valid/acc: 0.3717
Epoch: 2000   train/loss: 52.138   valid/loss: 38.4875   train/acc: 0.5105   valid/acc: 0.3186
Epoch: 2200   train/loss: 50.9937   valid/loss: 40

Test accuracy: 0.42342342342342343
Progress:  0.426
Number of training circuits: 191   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267


Epoch: 200   train/loss: 63.8467   valid/loss: 39.0855   train/acc: 0.3455   valid/acc: 0.3009
Epoch: 400   train/loss: 59.0377   valid/loss: 41.0798   train/acc: 0.4136   valid/acc: 0.2832
Epoch: 600   train/loss: 61.3152   valid/loss: 39.97   train/acc: 0.3927   valid/acc: 0.2035
Epoch: 800   train/loss: 57.735   valid/loss: 39.2268   train/acc: 0.4031   valid/acc: 0.2566
Epoch: 1000   train/loss: 58.0042   valid/loss: 39.6803   train/acc: 0.377   valid/acc: 0.2743
Epoch: 1200   train/loss: 54.3083   valid/loss: 40.0552   train/acc: 0.4398   valid/acc: 0.2655
Epoch: 1400   train/loss: 52.7487   valid/loss: 42.6692   train/acc: 0.445   valid/acc: 0.2743
Epoch: 1600   train/loss: 52.9203   valid/loss: 43.1978   train/acc: 0.4607   valid/acc: 0.3009
Epoch: 1800   train/loss: 51.5592   valid/loss: 41.4444   train/acc: 0.4921   valid/acc: 0.292
Epoch: 2000   train/loss: 50.6048   valid/loss: 41.739   train/acc: 0.5236   valid/acc: 0.3186
Epoch: 2200   train/loss: 51.0469   valid/loss: 41.

Test accuracy: 0.4144144144144144
Progress:  0.429
Number of training circuits: 192   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267
